# Junos PyEZ
PyEZ is a Python library that enables administration and automation of Junos devices. It is an open source project maintained and supported by Juniper Networks with contributions from the user community. The Junos PyEZ project is hosted on GitHub at https://github.com/Juniper/py-junos-eznc. PyEZ provides an abstraction layer built on top of the NETCONF protocol (requires vendor-agnostic *ncclient* library). Rather than returning RPC responses as XML, PyEZ utilizes the *lxml* library to directly return XML-specific Python data structures. 

pip stand for "Pip Installs Packages" or "Pip Installs Python". pip is a package management system used to find, install and manage Python packages and many packages can be found in the PyPI (Python Package Index) repository. There are currently 70000 packages: https://pypi.python.org/pypi. pip can be used to find packages in PyPI and to install them. To install Junos PyEZ execute the command (and can optionally upgrade with the -U argument):

    pip install junos-eznc 

at a root shell prompt to install the latest stable release of Junos PyEZ. 

## NETCONF Device Connectivity
PyEZ requires NETCONF to be enabled on the target device. The configuration required is:

    seanw@vsrx1> edit 
    Entering configuration mode

    [edit]
    seanw@vsrx2# set system services netconf ssh port 830 
    

To enable netconf tracing and logs, configure the following in a device.

    [edit]
    seanw@vsrx1# load set terminal 
    [Type ^D at a new line to end input]
    set system services netconf traceoptions file netconf.log 
    set system services netconf traceoptions file size 3m
    set system services netconf traceoptions file files 20
    set system services netconf traceoptions file world-readable 
    set system services netconf traceoptions flag all
    load complete
    
    [edit]
    seanw@vsrx1# commit and-quit 
    commit complete
    Exiting configuration mode    

NETCONF-over-SSH listens on TCP port 830 by default and operates over IPv4 and IPv6. Below shows verification of the NETCONF session.

    seanw@Seans-iMac:~$ ssh -p 830 -s seanw@vsrx1 netconf
    <!-- No zombies were killed during the creation of this user interface -->
    <!-- user seanw, class j-super-user -->
    <hello xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
      <capabilities>
        <capability>urn:ietf:params:netconf:base:1.0</capability>
        <capability>urn:ietf:params:netconf:capability:candidate:1.0</capability>
        <capability>urn:ietf:params:netconf:capability:confirmed-commit:1.0</capability>
        <capability>urn:ietf:params:netconf:capability:validate:1.0</capability>
        <capability>urn:ietf:params:netconf:capability:url:1.0?scheme=http,ftp,file</capability>
        <capability>urn:ietf:params:xml:ns:netconf:base:1.0</capability>
        <capability>urn:ietf:params:xml:ns:netconf:capability:candidate:1.0</capability>
        <capability>urn:ietf:params:xml:ns:netconf:capability:confirmed-commit:1.0</capability>
        <capability>urn:ietf:params:xml:ns:netconf:capability:validate:1.0</capability>
        <capability>urn:ietf:params:xml:ns:netconf:capability:url:1.0?protocol=http,ftp,file</capability>
        <capability>http://xml.juniper.net/netconf/junos/1.0</capability>
        <capability>http://xml.juniper.net/dmi/system/1.0</capability>
      </capabilities>
      <session-id>3596</session-id>
    </hello>
    ]]>]]>

Input the following RPC for the CLI op command 'show version':

    <rpc>
    <get-software-information/>
    </rpc>
    
The RPC reply is encoded in **`<rpc-reply>`** tags:

    <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/15.1X49/junos">
    <software-information>
    <host-name>vsrx1</host-name>
    <product-model>vsrx</product-model>
    <product-name>vsrx</product-name>
    <junos-version>15.1X49-D75.5</junos-version>
    <package-information>
    <name>junos</name>
    <comment>JUNOS Software Release [15.1X49-D75.5]</comment>
    </package-information>
    </software-information>
    </rpc-reply>
    ]]>]]>

Input the following RPC to close the NETCONF session:

    <rpc>
    <close-session/>
    </rpc>

The following reply should be recieved:

    <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/15.1X49/junos">
    <ok/>
    </rpc-reply>
    ]]>]]>
    <!-- session end at 2017-02-04 04:20:10 UTC -->   

    seanw@vsrx1> show log netconf.log 
    Feb  4 04:19:52 Started tracing session: 3982
    Feb  4 04:19:54 [3982] Incoming:
    Feb  4 04:19:54 [3982] Incoming:
    Feb  4 04:20:02 [3982] Incoming: <rpc>
    Feb  4 04:20:02 [3982] Incoming: <get-software-information/>   
    Feb  4 04:20:03 [3982] Incoming: </rpc>    
    Feb  4 04:20:02 [3982] Outgoing: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/15.1X49/junos">
    Feb  4 04:20:02 [3982] Outgoing: <software-information>
    Feb  4 04:20:02 [3982] Outgoing: <host-name>vsrx1</host-name>
    Feb  4 04:20:02 [3982] Outgoing: <product-model>vsrx</product-model>
    Feb  4 04:20:02 [3982] Outgoing: <product-name>vsrx</product-name>
    Feb  4 04:20:02 [3982] Outgoing: <junos-version>15.1X49-D75.5</junos-version>
    Feb  4 04:20:02 [3982] Outgoing: <package-information>
    Feb  4 04:20:02 [3982] Outgoing: <name>junos</name>
    Feb  4 04:20:02 [3982] Outgoing: <comment>JUNOS Software Release [15.1X49-D75.5]</comment>
    Feb  4 04:20:02 [3982] Outgoing: </package-information>
    Feb  4 04:20:02 [3982] Outgoing: </software-information>
    Feb  4 04:20:03 [3982] Outgoing: </rpc-reply>
    Feb  4 04:20:03 [3982] Outgoing: ]]>]]>
    Feb  4 04:20:10 [3982] Incoming: <rpc>
    Feb  4 04:20:10 [3982] Incoming: <close-session/>    
    Feb  4 04:20:10 [3982] Incoming: </rpc>    
    Feb  4 04:20:10 [3982] Outgoing: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/15.1X49/junos">
    Feb  4 04:20:10 [3982] Outgoing: <ok/>
    Feb  4 04:20:10 [3982] Outgoing: </rpc-reply>
    Feb  4 04:20:10 [3982] Outgoing: ]]>]]>
    Feb  4 04:20:10 [3982] Outgoing: <!-- session end at 2017-02-04 04:20:10 UTC -->

### Device Instance
The PyEZ library provides a *jnpr.junos.Device* class to represent a Junos device being accessed by the PyEZ library. The first step in using the library is to instantiate an instance of this class with the parameters specific to the Junos device. The first command imports the *jnpr.junos* Python package and copies the *Device* name into the local namespace, allowing you to simply reference **Device()**. The second line calls the *Device* class object with the **Device()** syntax creates a new instance object that is referenced with the variable named vsrx1. The class *Device* is defined in the module device (device.py) in the package *jnpr.junos*.

Device authentiation can be done using password or SSH keys. For interactive scripts the user can be prompted for the parameters using the **input()** function and storing the results in a variable. Likewise the password can be prompted for and not shown on the screen using:

    from getpass import getpass
    password = getpass()

In [1]:
from jnpr.junos import Device
vsrx1 = Device(host = 'vsrx1', user = 'demo', password = 'Juniper')

### Connecting
The NETCONF connection is only made using the information stored in the device instance attributes when the **open()** method is invoked. Likewise you can check if a NETCONF session using *connected*.

In [2]:
vsrx1.open()

Device(vsrx1)

In [3]:
vsrx1.connected

True

The **close()** method cleanly shuts down the NETCONF session that was initiated by the successful execution of the **open()** method. Calling the **close()** method does not destroy the device instance; it just closes its NETCONF session. Calling an RPC after an instance has been closed causes a *jnpr.junos.exception.ConnectClosedError* exception to be raised. A NETCONF connection will (normally) be closed after completing the ‘idle’ time of the session defined in configuration. However, since SSH sessions are finite resources (and can be limited by configuration), it is important to close the NETCONF session.

In [4]:
vsrx1.close()

In [5]:
vsrx1.connected

False

By wrapping the **open()** method in a try/except block then exceptions can be handled gracefully and requires the *jnpr.junos.exception* module to be imported.

In [6]:
import jnpr.junos.exception

vsrx1 = Device(host='vsrx1', user='demo', password='bad_pass')

try:
    vsrx1.open()
except jnpr.junos.exception.ConnectError as err:
    print "Error: " + repr(err)

Error: ConnectAuthError(vsrx1)


### Fact Gathering
By default, the PyEZ library gathers basic information about the device during the **open()** call and stores this information in a Python dictionary. This dictionary can be easily accessed with the *facts* attribute.

In [7]:
from pprint import pprint
vsrx1 = Device(host='vsrx1', user='demo', password='Juniper')
vsrx1.open()

Device(vsrx1)

In [8]:
pprint(vsrx1.facts)

{'2RE': False,
 'HOME': '/var/home/demo',
 'RE0': {'last_reboot_reason': '0x4000:VJUNOS reboot ',
         'model': 'VSRX-S',
         'status': 'OK',
         'up_time': '7 hours, 45 minutes, 2 seconds'},
 'domain': None,
 'fqdn': 'vsrx1',
 'hostname': 'vsrx1',
 'ifd_style': 'CLASSIC',
 'model': 'VSRX',
 'personality': 'UNKNOWN',
 'serialnumber': 'BABF6658EFA5',
 'srx_cluster': False,
 'switch_style': 'NONE',
 'vc_capable': False,
 'version': '15.1X49-D75.5',
 'version_info': junos.version_info(major=(15, 1), type=X, minor=(49, 'D', 75), build=5)}


In [9]:
vsrx1.facts['model']

'VSRX'

In [10]:
vsrx1.facts['RE0']['up_time']

'7 hours, 45 minutes, 2 seconds'

In [11]:
vsrx1.facts['serialnumber']

'BABF6658EFA5'

In [12]:
vsrx1.facts['version']

'15.1X49-D75.5'

Some facts can be refreshed using the **facts_refresh()** method before accessing the fact again (such as uptime).

In [13]:
vsrx1.facts_refresh()

In [14]:
vsrx1.facts['RE0']['up_time']

'7 hours, 49 minutes, 14 seconds'

In [15]:
vsrx1.close

<bound method Device.close of Device(vsrx1)>

### Device Probe
To check if NETCONF can be reached the device can be probed. This can be useful in re-connecting after a reboot. The default timeout is 10 seconds.

In [16]:
vsrx1.probe(timeout=30)

True